In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 12.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import time
import scipy.stats
from torch.utils.data import TensorDataset, DataLoader
import optuna
from torch import optim
import copy
import torch.nn.functional as F

Загрузка данных

In [2]:
df = pd.read_csv('daily_accidents_hol_dw_week.csv', parse_dates=['CRASH DATE'])
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'])
df.set_index('CRASH DATE', inplace=True)
features = ['CRASH_COUNT', 'is_weekend', 'month', 'is_holiday']
df_features = df[features]

Масштабирование данных

In [ ]:
scaler_X = MinMaxScaler()
scaled_features = scaler_X.fit_transform(df_features)

scaler_y = MinMaxScaler()
scaled_target = scaler_y.fit_transform(df[['CRASH_COUNT']])

Создание датасета

In [4]:
def create_multivariate_dataset(X_data, y_data, time_step=1):
    X, y = [], []
    for i in range(len(X_data) - time_step):
        X.append(X_data[i:i+time_step])
        y.append(y_data[i+time_step])
    return np.array(X), np.array(y)

time_step = 60
X, y = create_multivariate_dataset(scaled_features, scaled_target, time_step)


Создание  обучающей и тестовой выборки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

if y_train.ndim == 1:
    y_train = y_train.unsqueeze(-1)
    y_test = y_test.unsqueeze(-1)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Определение моделей

In [6]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size, num_layers, dropout, bidirectional):
        super(LSTMModel, self).__init__()
        self.bidirectional = bidirectional
        self.lstm = nn.LSTM(
            input_size,
            hidden_layer_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
            bidirectional=bidirectional
        )
        self.fc = nn.Linear(hidden_layer_size * (2 if bidirectional else 1), output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out


class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size, num_layers, dropout, bidirectional):
        super(GRUModel, self).__init__()
        self.bidirectional = bidirectional
        self.gru = nn.GRU(
            input_size,
            hidden_layer_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
            bidirectional=bidirectional
        )
        self.fc = nn.Linear(hidden_layer_size * (2 if bidirectional else 1), output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out


Функция подбора гиперпараметров LSTM

In [ ]:
def objective_lstm(trial):
    hidden_size = trial.suggest_int("hidden_layer_size", 32, 256)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    learning_rate = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    bidirectional = trial.suggest_categorical("bidirectional", [True, False])

    model = LSTMModel(
        input_size=4,
        hidden_layer_size=hidden_size,
        output_size=1,
        num_layers=num_layers,
        dropout=dropout,
        bidirectional=bidirectional
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    model.train()
    for epoch in range(20):
        for inputs, labels in train_loader:
            inputs, labels = inputs, labels
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs, labels
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

    return test_loss / len(test_loader)


Подбор гиперпараметров LSTM

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective_lstm, n_trials=30)

print("Best parameters:", study.best_params)

[I 2025-05-05 14:10:01,496] A new study created in memory with name: no-name-235e72c9-45ff-4d07-868e-f7c8338764d7
[I 2025-05-05 14:25:06,065] Trial 0 finished with value: 0.013810363533938753 and parameters: {'hidden_layer_size': 251, 'num_layers': 3, 'lr': 0.0001490076294330111, 'dropout': 0.20175387711037523, 'bidirectional': False}. Best is trial 0 with value: 0.013810363533938753.
[I 2025-05-05 14:32:00,100] Trial 1 finished with value: 0.0008318287878425728 and parameters: {'hidden_layer_size': 84, 'num_layers': 3, 'lr': 0.00028879178674107885, 'dropout': 0.11339128170192025, 'bidirectional': True}. Best is trial 1 with value: 0.0008318287878425728.
[I 2025-05-05 14:56:41,497] Trial 2 finished with value: 0.02896846721655336 and parameters: {'hidden_layer_size': 189, 'num_layers': 3, 'lr': 0.0012727052023361902, 'dropout': 0.2580927562182407, 'bidirectional': True}. Best is trial 1 with value: 0.0008318287878425728.
[I 2025-05-05 14:59:31,632] Trial 3 finished with value: 0.001429

Best parameters: {'hidden_layer_size': 167, 'num_layers': 2, 'lr': 0.0005295329548379808, 'dropout': 0.18601878751913226, 'bidirectional': True}


Функция подбора гиперпараметорв GRU

In [ ]:
def objective_gru(trial):
    hidden_size = trial.suggest_int("hidden_layer_size", 32, 256)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    learning_rate = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    bidirectional = trial.suggest_categorical("bidirectional", [True, False])

    model = GRUModel(
        input_size=4,
        hidden_layer_size=hidden_size,
        output_size=1,
        num_layers=num_layers,
        dropout=dropout,
        bidirectional=bidirectional
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    model.train()
    for epoch in range(20):
        for inputs, labels in train_loader:
            inputs, labels = inputs, labels
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs, labels
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

    return test_loss / len(test_loader)


Подбор гиперпараметров GRU

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective_gru, n_trials=30)

print("Best parameters:", study.best_params)

[I 2025-05-05 20:48:28,165] A new study created in memory with name: no-name-2531cb84-c815-4ae5-9fb7-affacecb3f90
[I 2025-05-05 21:11:59,517] Trial 0 finished with value: 0.00251309417493256 and parameters: {'hidden_layer_size': 247, 'num_layers': 2, 'lr': 0.0038066450082737415, 'dropout': 0.06639099762016759, 'bidirectional': True}. Best is trial 0 with value: 0.00251309417493256.
[I 2025-05-05 21:13:30,305] Trial 1 finished with value: 0.0017606234258233473 and parameters: {'hidden_layer_size': 85, 'num_layers': 1, 'lr': 0.0038245008617689216, 'dropout': 0.3443041596069208, 'bidirectional': False}. Best is trial 1 with value: 0.0017606234258233473.
[I 2025-05-05 21:15:48,267] Trial 2 finished with value: 0.0007123792311176658 and parameters: {'hidden_layer_size': 150, 'num_layers': 1, 'lr': 0.00392972128380435, 'dropout': 0.3853362859935563, 'bidirectional': False}. Best is trial 2 with value: 0.0007123792311176658.
[I 2025-05-05 21:24:53,466] Trial 3 finished with value: 0.002497450

Best parameters: {'hidden_layer_size': 150, 'num_layers': 1, 'lr': 0.00392972128380435, 'dropout': 0.3853362859935563, 'bidirectional': False}


Инициализация моделей

In [ ]:
input_size = 4
output_size = 1

lstm_model = LSTMModel(input_size, output_size=output_size, hidden_layer_size=167, num_layers=2, dropout=0.18601878751913226, bidirectional=False)
gru_model = GRUModel(input_size, output_size=output_size, hidden_layer_size=150, num_layers=1, dropout=0.3853362859935563, bidirectional=False)

criterion = nn.MSELoss()
lstm_optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.0005295329548379808)
gru_optimizer = torch.optim.Adam(gru_model.parameters(), lr=0.00392972128380435)

Функция обучения

In [62]:
def train_model(model, optimizer, X_train, y_train, epochs=200):
    model.train()
    for epoch in range(epochs):
        output = model(X_train)
        loss = criterion(output, y_train.view(-1, 1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Обучение LSTM

In [63]:
start_time = time.time()
train_model(lstm_model, lstm_optimizer, X_train, y_train, epochs=200)
lstm_time = time.time() - start_time

Обучение GRU

In [13]:
start_time = time.time()
train_model(gru_model, gru_optimizer, X_train, y_train)
gru_time = time.time() - start_time

Прогнозирование и возвращение оригинального масштаба

In [ ]:
def predict(model, X_test):
    model.eval()
    with torch.no_grad():
        predicted = model(X_test)
    return predicted

lstm_predicted = predict(lstm_model, X_test)
gru_predicted = predict(gru_model, X_test)

lstm_predicted = scaler_y.inverse_transform(lstm_predicted.numpy())
gru_predicted = scaler_y.inverse_transform(gru_predicted.numpy())
y_test_rescaled = scaler_y.inverse_transform(y_test.numpy())

Функция вычисление корреляции

In [ ]:
def check_nan(y_true, y_pred):
    if np.any(np.isnan(y_true)) or np.any(np.isnan(y_pred)):
        print("Есть NaN в данных!")
        return True
    return False

def calculate_correlation(y_true, y_pred):
    if check_nan(y_true, y_pred):
        return np.nan
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    corr, _ = scipy.stats.pearsonr(y_true, y_pred)
    return corr

Оценка по метрикам

In [65]:
def evaluate_model(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    correlation = calculate_correlation(y_true, y_pred)
    return rmse, mae, r2, correlation

lstm_rmse, lstm_mae, lstm_r2, lstm_corr = evaluate_model(y_test_rescaled, lstm_predicted)
gru_rmse, gru_mae, gru_r2, gru_corr = evaluate_model(y_test_rescaled, gru_predicted)

print(f"LSTM RMSE: {lstm_rmse}, MAE: {lstm_mae}, R²: {lstm_r2}, Correlation: {lstm_corr}")
print(f"GRU RMSE: {gru_rmse}, MAE: {gru_mae}, R²: {gru_r2}, Correlation: {gru_corr}")

LSTM RMSE: 31.89522087627069, MAE: 24.80636978149414, R²: 0.1845739483833313, Correlation: 0.48860105872154236
GRU RMSE: 28.015986647135954, MAE: 21.695613861083984, R²: 0.37086302042007446, Correlation: 0.6250824332237244


Время на обучение моделей

In [70]:
print(f"Время обучения LSTM: {lstm_time} секунд")

Время обучения LSTM: 3651.713054895401 секунд


In [71]:
print(f"Время обучения GRU: {gru_time} секунд")

Время обучения GRU: 2592.8408772945404 секунд
